<center>
<h1> TP 1-2 de Recherche opérationnelle </h1>
<h1> Année 2020-2021 - 2e année département Sciences du Numérique </h1>
<h1> EL BOUCHIBTI Aymane</h1>
<h1> AKBY Amine </h1>    
</center>

# Applications en optimisation pour l’e-commerce

# Cas particulier 1
 
#### Objectif :

L’objectif de ce problème est de minimiser le coût de stockage des différentes commandes des clients.

#### Données :

— Fluides commandés par les clients.

— Stocks de fluides disponibles par magasins.

— Coût unitaire par fluide pour les différents magasins.

#### Variables :

La variable de décision qu'on souhaite déterminer est la répartition des différents fluides dans chaque magasin.

#### Contraintes :

— La somme des quantités prises pour un fluide dans l'ensemble des magasins doit être égale à la quantité demandée de ce fluide dans l'ensemble des demandes.

— Les quantités prises de chaque fluide doivent être inférieures où égales aux quantités disponibles dans les magasins.

In [17]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
A = [2 0; 1 3] # fluides demandés par commande
B = [2.5 1; 1 2; 2 1] #Stocks de fluides par magasin
C = [1 1; 2 3; 3 2] #Couts unitaires par entrepôt

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, M[1:3,1:2] >= 0)

# define objective function
@objective(model, Min, sum(M.*C))

# define constraints
for i in 1:3
    for j in 1:2
        @constraint(model, M[i,j] - B[i,j] <= 0)
    end
end

#Bonne répartition des fluides 1 et 2 sur les magasins
@constraint(model, sum(M[:,1]) - sum(A[:,1]) == 0)
@constraint(model, sum(M[:,2]) - sum(A[:,2]) == 0)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 2 (-6) rows, 6 (0) columns and 6 (-6) elements
0  Obj 2.6999999 Primal inf 5.099998 (2)
2  Obj 9.5
Optimal - objective value 9.5
After Postsolve, objective 9.5, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 9.5 - 2 iterations time 0.002, Presolve 0.00
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



In [20]:
# print solution
println("Solution obtenue:")
println("\t Coût = $(objective_value(model))")
for i in 1:3
    for j in 1:2
        println("\t la répartition du fluide $j sur le magasin $i = $(value(M[i,j]))")
    end
end

Solution obtenue:
	 Coût = 9.5
	 la répartition du fluide 1 sur le magasin 1 = 2.5
	 la répartition du fluide 2 sur le magasin 1 = 1.0
	 la répartition du fluide 1 sur le magasin 2 = 0.5
	 la répartition du fluide 2 sur le magasin 2 = 1.0
	 la répartition du fluide 1 sur le magasin 3 = 0.0
	 la répartition du fluide 2 sur le magasin 3 = 1.0


#### Analyse des résultats :

Pour cette exemple, on a 3 du fluide 1 et 3 du fluide 2 à stocker dans les magasins.

Le premier magasin est moins cher pour les deux fluides que les deux autres magasins.
Donc l'optimiseur va préviligier ce magasin pour stocker les fluides jusqu'à qu'il soit remplit.

Pour les deux autres magasins, l'optimiseur va stocker chaque quantité de fluide dans le magasin le moins cher pour ce fluide.
Donc, il va stocker le reste du fluide 1 dans le magasin 2 et le reste du fluide 2 dans le magasin 3.

La répartition donnée par l'optimiseur est donc logique.

# Cas particulier 2
 
Ce problème est similaire au cas particulier 1.

Le seul point différent est qu'on raisonne sur des produits au lieu des fluides. Ce qui fait que la quantité des différents produits dans les magasins doit être dans N car on ne peut pas diviser un produit pour le stocker en deux différents magasins.
 
#### Objectif :

L’objectif de ce problème est le même que pour le cas particulier 1.

#### Données :

— On conserve les données du cas particulier 1.

#### Variables :

La répartition des différents fluides dans chaque magasin.

#### Contraintes :

— On conserve les contraintes du cas particulier 1.
— On ajoute le fait que la variable décision doit être dans N.

In [22]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
A = [2 0; 1 3] # produits demandés par commande
B = [2.5 1; 1 2; 2 1] #Stocks de produits par magasin
C = [1 1; 2 3; 3 2] #Couts unitaires par entrepôt

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, M[1:3,1:2] >= 0)

# define objective function
@objective(model, Min, sum(M.*C))

# define constraints
for i in 1:3
    for j in 1:2
        @constraint(model, M[i,j] - B[i,j] <= 0)
        @constraint(model, M[i,j] in MOI.Integer())
    end
end

#Bonne répartition des produits dans les magasins
@constraint(model, sum(M[:,1]) - sum(A[:,1]) == 0)
@constraint(model, sum(M[:,2]) - sum(A[:,2]) == 0)

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 9.5 - 0.00 seconds
Cgl0004I processed model has 2 rows, 6 columns (6 integer (3 of which binary)) and 6 elements
Cbc0012I Integer solution of 10 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 10, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 10 to 10
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after add

In [25]:
# print solution
println("Solution obtenue:")
println("\t Coût = $(objective_value(model))")
for i in 1:3
    for j in 1:2
        println("\t la répartition du produit $j sur le magasin $i = $(value(M[i,j]))")
    end
end

Solution obtenue:
	 Coût = 10.0
	 la répartition du produit 1 sur le magasin 1 = 2.0
	 la répartition du produit 2 sur le magasin 1 = 1.0
	 la répartition du produit 1 sur le magasin 2 = 1.0
	 la répartition du produit 2 sur le magasin 2 = 1.0
	 la répartition du produit 1 sur le magasin 3 = 0.0
	 la répartition du produit 2 sur le magasin 3 = 1.0


#### Analyse des résultats :

Par le même raisonnement du cas particulier 1, le magasin 1 sera préviligier jusqu'à qu'il soit plein ou ce qui reste est < 1. Même chose pour les autres magasins.

La répartition donnée par l'optimiseur est donc logique.

Le coût trouvée par l'optimiseur est 10.0. En effet, on a ajouté une contrainte pour la répartition des produits, ce qui fait qu'on sait que le coût sera supérieur au coût du cas particulier 1 qui était 9.5 .

# Cas particulier 3
 
On prend en compte les coûts d'expédition des différents colis des magasins aux clients.
 
#### Objectif :

L'objectif est de minimiser en même temps le coût de stockage des différents produits dans les magasins et le coût d'expédition des différents colis des magasins aux clients.

#### Données :

— Produits commandés par les clients.

— Stocks de produits disponibles par magasins.

— Coût unitaire par produit pour les différents magasins.

— Coûts d'expédition d'un colis de chaque magasin vers chaque client.

#### Variables :

— La répartition des différents produits commandés par chaque client dans chaque magasin.

— Une matrice binaire Z qui indique si un client à un colis stockés dans un magasin. 

#### Contraintes :

— La quantité commandé par un client d'un certain produit doit être égale à la même quantité stockés dans tous les magasins.

— Les quantités prises de chaque produit doit être inférieures où égales aux quantités disponibles dans les magasins.

— La case Z[i,j] est égal à 1 si un magasin i à un colis stockés d'un client j.

In [27]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP

# data
A = [2 0; 1 3] # produits demandés par commande
B = [2 1; 1 2; 2 1] #Stocks de produits par magasin
C = [1 1; 2 3; 3 2] #Couts unitaires par entrepôt
D = [1 0 0; 0 2 1] #Couts d'expédition d'un colis

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, M[1:3,1:2,1:2] >= 0)
@variable(model, Z[1:3,1:2], Bin)

# define objective function
@objective(model, Min, sum((M[:,:,1] + M[:,:,2]).*C + transpose(D).*Z))

# define constraints
for i in 1:3
    for j in 1:2
        @constraint(model, M[i,j,1] + M[i,j,2] - B[i,j] <= 0)
        @constraint(model, M[i,j,1] in MOI.Integer())
        @constraint(model, M[i,j,2] in MOI.Integer())
    end
end

#Bonne répartition des produits 1 et 2 sur les magasins
for i in 1:2
    for j in 1:2
        #La quantité d'un produit j commandés par un client i
        # doit être égale à la quantité de j commandés par
        # i stockés dans tous les magasins.
        @constraint(model, sum(M[:,j,i]) == A[i,j])
    end
end

# La matrice Z est binaire.
# Si Z[i,j] = 1, le magasin i va expédier un colier au demandeur j
# sinon si Z[i,j] = 0, le magasin i n'a pas de colis pour j
for i in 1:3
    for j in 1:2
        @constraint(model, sum(M[i,:,j])/(sum(A[j,:])) <= Z[i,j])
        @constraint(model,Z[i,j] <= sum(M[i,:,j]))
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 11.25 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 3 substitutions
Cgl0004I processed model has 14 rows, 13 columns (13 integer (10 of which binary)) and 37 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 14, took 1 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 3 variables fixed on reduced cost
Cuts at root node changed objective from 13.5 to 14
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 s

In [31]:
# print solution
println("Solution obtenue:")
println("\t Coût = $(objective_value(model))")
for i in 1:3
    for j in 1:2
        for k in 1:2
            println("\t la répartition du produit $j sur le magasin $i pour le client $k = $(value(M[i,j,k]))")
        end
    end
end
for i in 1:3
    for j in 1:2
        if value(Z[i,j]) == 1
            println("\t Le magasin $i stock un colis du client $j")
        end
    end
end

Solution obtenue:
	 Coût = 14.0
	 la répartition du produit 1 sur le magasin 1 pour le client 1 = 1.0
	 la répartition du produit 1 sur le magasin 1 pour le client 2 = 1.0
	 la répartition du produit 2 sur le magasin 1 pour le client 1 = 0.0
	 la répartition du produit 2 sur le magasin 1 pour le client 2 = 1.0
	 la répartition du produit 1 sur le magasin 2 pour le client 1 = 1.0
	 la répartition du produit 1 sur le magasin 2 pour le client 2 = 0.0
	 la répartition du produit 2 sur le magasin 2 pour le client 1 = 0.0
	 la répartition du produit 2 sur le magasin 2 pour le client 2 = 1.0
	 la répartition du produit 1 sur le magasin 3 pour le client 1 = 0.0
	 la répartition du produit 1 sur le magasin 3 pour le client 2 = 0.0
	 la répartition du produit 2 sur le magasin 3 pour le client 1 = 0.0
	 la répartition du produit 2 sur le magasin 3 pour le client 2 = 1.0
	 Le magasin 1 stock un colis du client 1
	 Le magasin 1 stock un colis du client 2
	 Le magasin 2 stock un colis du client 1
	 

#### Analyse des résultats :

L'analyse devient un peut compliquée car in doit tenir compte de différents paramêtres.

Cependant on pourrait confirmer que le coût parraît logique car il est supérieur au coût du cas particulier 2.

On peut aussi remarqué que l'optimiseur évite de stocker un colis d'un client dans un magasin où le coût est trop cher.

# Problème général

On modélise un problème de livraison des commandes des clients par les livreurs des différents magasins.

#### Objectif :

L'objectif est de minimiser les trajets de livraisons des commandes pour les livreurs des différents magasins.

#### Données :

— Nombre de clients.

— Nombre de magasins.

— Nombre de produits.

— Produits commandés par clients.

— Stocks de produits disponibles par magasins.

— Ensemble des noeuds des différents sites (clients + magasins).

— Distance du trajet à parcourir entre les différents noeuds.

#### Variables :

— La répartition des différents produits commandés par chaque client dans chaque magasin.

— Une matrice binaire Z qui indique si le livreur de chaque magasin passe entre deux noeuds du site.

Z[:,:,k] est la matrice de déplacement du livreur de magasin k.
La dimension de Z[:,:,k] est (le nombre de client +1) x (le nombre de client +1). En effet, on ajoute 1 pour prendre en compte le magazin.

#### Contraintes :

— La case Z[i,j,k] est égal à 1 si le livreur du magasin k passe du noeud i au noeud j.

— Chaque magasin dispose de son propre livreur qui sera en charge de livrer en une seule tournée tous les produits qui proviennent de son magasin. Pour chaque magasin qui expédie au moins 1 produit, son livreur/camion débute sa tournée au magasin, visite une seule fois chacun des clients qu’il doit servir et retourne au magasin en fin de tournée. (Matrice Z[:,:,k] du magasin k aura un 1 par ligne et par colonne)

— chaque commande doit être satisfaite en totalité.

— aucun magasin ne peut faire livrer plus de produits qu’il n’en possède en stock.

— Z[i,j,k] = 1 si le livreur du magasin k passe du client ou du magazin i vers le client ou le magazin j.

In [44]:
# fonctionne pour JuMP version 0.21.5
using Cbc
using JuMP
using LinearAlgebra

include("2_4.jl")
inputfilepath = "data_2-4"
inputfilename = "Data_test_5_1_2.txt"

## Data
# nb_dem   : Nombre de demandeurs.
# nb_prod  : Nombre de produits
# nb_mag   : Nombre de magasins.
# nb_noeud : Nombre de noeuds.
# S        : Smp Stock de produit p dans le magasin M
# Q        : Qdp Quantité de produit p dans la commande d
# R        : Rij valeur de l'arc représentant la distance à parcourir/temps de trajet entre les sites i et j.
# N        : L'ensemble des noeuds.
nb_dem, nb_prod, nb_mag, nb_noeuds, S, Q, R = read_data_24(inputfilepath, inputfilename);

# set optimizer
model = Model(Cbc.Optimizer)

# define variables
@variable(model, M[1:nb_mag, 1:nb_prod, 1:nb_dem] >= 0)
@variable(model, Z[1:(1+nb_dem),1:(1+nb_dem),1:nb_mag], Bin)

# define objective function
@objective(model, Min, sum(reshape(sum(Z[2:(1+nb_dem),2:(1+nb_dem),:],dims = 3),nb_dem,nb_dem).*reduce(hcat,R[1+nb_mag:nb_dem+nb_mag])[1+nb_mag:nb_dem+nb_mag,:])
                           +sum((Z[2:1+nb_dem,1,:] + Z[1,2:1+nb_dem,:]).*reduce(hcat,R[1:nb_mag])[1+nb_mag:nb_mag+nb_dem,:]))

#Contrainte2
for i in 1:nb_mag
    @constraint(model, sum(Z[1,2:1+nb_dem,i]) <= sum(M[i,:,:]))
    @constraint(model, sum(M[i,:,:]) /(1 + sum(S[i][:]))<= sum(Z[1,2:1+nb_dem,i]))
    @constraint(model, tr(Z[:,:,i])==0)
    for j in 1:1+nb_dem
        @constraint(model, sum(Z[j,:,i]) <= sum(M[i,:,:]))
        @constraint(model, sum(M[i,:,:])/(1+sum(S[i][:]))<= sum(Z[j,:,i]))
        @constraint(model, sum(Z[:,j,i]) <= sum(M[i,:,:]))
        @constraint(model, sum(M[i,:,:])/(1+sum(S[i][:]))<= sum(Z[:,j,i]))
    end
     for j in 1:nb_dem 
        @constraint(model, sum(Z[j,1:j,i])+sum(Z[1:j,j,i]) <= 1)
    end
    @constraint(model, sum(Z[2:1+nb_dem,1,i]) <= sum(M[i,:,:]))
    @constraint(model, sum(M[i,:,:]) /(1 + sum(S[i][:]))<= sum(Z[2:1+nb_dem,1,i]))
end

#Contrainte3
for i in 1:nb_dem
    for j in 1:nb_prod
        @constraint(model, sum(M[:,j,i]) == Q[i][j])
    end
end

# Constrainte5
for i in 1:nb_mag
    for j in 1:nb_prod
        @constraint(model,sum(M[i,j,:]) <= S[i][j])
    end
end

# Z
for i in 1:nb_dem
    for j in 1:nb_mag
        @constraint(model,sum(Z[i+1,:,j]) <= sum(M[j,:,i]))
        @constraint(model,sum(Z[:,i+1,j]) <= sum(M[j,:,i]))
        @constraint(model,sum(M[j,:,i])/(1+sum(Q[i][:])) <= sum(Z[:,i+1,j]))
        @constraint(model,sum(M[j,:,i])/(1+sum(Q[i][:])) <= sum(Z[i+1,:,j]))
    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 7.26068 - 0.00 seconds
Cgl0002I 6 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 22 rows, 30 columns (30 integer (30 of which binary)) and 116 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 14, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 14 to 14
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0

In [40]:
JuMP.value.(Z)

6×6×1 Array{Float64,3}:
[:, :, 1] =
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  0.0

#### Analyse des résultats :

On analyse les résultats pour le test 5_1_2 par exemple.

On remarque que la matrice Z est bien structurée.

Chemin du livreur :

Magasin -> Client 1 -> Client 4 -> Client 5 -> Client 3 -> Client 2 -> Magasin.

Le livreur débute la livraison du magasin et passe une seul fois chez chaque client et retourne au magasin.

Cependant, c'est difficile de vérifier d'une manière simple que le chemin parcourue est le meilleur.